In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL to scrape
url = "https://uel.unipd.it/en/masters/"

# Fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the program names and their links
programs = []
links = []

# Assuming program names and links are within anchor tags inside a specific container
for link in soup.find_all('a', href=True):
    if 'master' in link['href']:  # Filter links containing 'master' (adjust as needed)
        program_name = link.get_text(strip=True)
        if program_name:  # Avoid empty program names
            programs.append(program_name)
            links.append(link['href'])

# Create a DataFrame
df = pd.DataFrame({
    'Program Name': programs,
    'Program Link': links
})



In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Fetch the webpage content
url = "https://uel.unipd.it/en/masters/"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Step 2: Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Step 3: Find the container with programs
    programs = soup.find_all('div', class_='master-object')

    # Lists to store data
    program_names = []
    program_links = []

    # Step 4: Extract program names and links
    for program in programs:
        # Find the link
        a_tag = program.find('a', href=True)
        if a_tag:
            link = a_tag['href']  # Extract href attribute
            program_links.append(link)

        # Find the program name
        h4_tag = program.find('h4')
        if h4_tag:
            name = h4_tag.get_text(strip=True)
            program_names.append(name)

    # Step 5: Store the data in a DataFrame
    data = pd.DataFrame({
        'Program Name': program_names,
        'Link': program_links
    })



In [9]:
data['Link'][3]

'https://uel.unipd.it/en/masters/idp-digital-implantology/'

In [10]:
data

,Program Name,Link
0,"CAT""Creative arts therapies"" to support resili...",https://uel.unipd.it/en/masters/cat-creative-a...
1,"NUESAHuman nutrition, education and food safety",https://uel.unipd.it/en/masters/nuesa-human-nu...
2,SEODental Sedation and Emergency,https://uel.unipd.it/en/masters/seo-dental-sed...
3,IDPDigital implantology,https://uel.unipd.it/en/masters/idp-digital-im...
4,PSITMOSPsychology of Traffic and Sustainable M...,https://uel.unipd.it/en/masters/psitmos-psycho...
5,AUAAdvanced urological anesthesia,https://uel.unipd.it/en/masters/aua-advanced-u...
6,GLCManagement of Skin Lesions and Difficult Wo...,https://uel.unipd.it/en/masters/glc-management...
7,NFCCForensic neuropsychology and clinical crim...,https://uel.unipd.it/en/masters/nfcc-forensic-...
8,RNResearch nursing,https://uel.unipd.it/en/masters/rn-research-nu...
9,CHDPaediatric Cardiology and Congenital Heart ...,https://uel.unipd.it/en/masters/chd-paediatric...


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Step 1: Set up Selenium WebDriver (adjust path to your chromedriver)
driver = webdriver.Chrome()
url = "https://uel.unipd.it/en/masters/"
driver.get(url)

# Wait until the page loads
wait = WebDriverWait(driver, 10)

# Lists to store program names and links
program_names = []
program_links = []

# Step 2: Extract data from all pages
while True:
    try:
        # Wait for the program container to load
        programs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.master-object')))

        # Extract program names and links from the current page
        for program in programs:
            # Extract program name
            name_element = program.find_element(By.TAG_NAME, 'h4')
            program_names.append(name_element.text)

            # Extract program link
            link_element = program.find_element(By.TAG_NAME, 'a')
            program_links.append(link_element.get_attribute('href'))

        # Step 3: Navigate to the next page
        next_button = driver.find_element(By.CSS_SELECTOR, '.pagination-next')
        if 'disabled' in next_button.get_attribute('class'):
            # If the "Next" button is disabled, break the loop
            break

        # Click the "Next" button
        ActionChains(driver).move_to_element(next_button).click(next_button).perform()
        time.sleep(2)  # Wait for the page to load
    except Exception as e:
        print(f"An error occurred: {e}")
        break

# Step 4: Close the browser
driver.quit()

# Step 5: Save the data to a CSV file
data = pd.DataFrame({
    'Program Name': program_names,
    'Link': program_links
})
data.to_csv('master_programs.csv', index=False)

print("Data extracted and saved to 'master_programs.csv'.")


An error occurred: Message: 

Data extracted and saved to 'master_programs.csv'.


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Step 1: Set up Selenium WebDriver
driver = webdriver.Chrome()
base_url = "https://uel.unipd.it/en/masters/page/{}/"

# Lists to store program names and links
program_names = []
program_links = []

# Step 2: Iterate through the pages
for page in range(1, 9):  # Loop from page 1 to page 8
    try:
        url = base_url.format(page)
        driver.get(url)
        print(f"Scraping page: {url}")
        
        # Wait for the page to load
        wait = WebDriverWait(driver, 10)
        programs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.master-object')))

        # Extract program names and links
        for program in programs:
            # Extract program name
            name_element = program.find_element(By.TAG_NAME, 'h4')
            program_names.append(name_element.text)

            # Extract program link
            link_element = program.find_element(By.TAG_NAME, 'a')
            program_links.append(link_element.get_attribute('href'))

        time.sleep(2)  # Optional: Add delay for smoother navigation
    except Exception as e:
        print(f"An error occurred on page {page}: {e}")
        break

# Step 3: Close the browser
driver.quit()

# Step 4: Save the data to a CSV file
data = pd.DataFrame({
    'Program Name': program_names,
    'Link': program_links
})


Scraping page: https://uel.unipd.it/en/masters/page/1/
Scraping page: https://uel.unipd.it/en/masters/page/2/
Scraping page: https://uel.unipd.it/en/masters/page/3/
Scraping page: https://uel.unipd.it/en/masters/page/4/
Scraping page: https://uel.unipd.it/en/masters/page/5/
Scraping page: https://uel.unipd.it/en/masters/page/6/
Scraping page: https://uel.unipd.it/en/masters/page/7/
Scraping page: https://uel.unipd.it/en/masters/page/8/


In [19]:
data.nunique()

Program Name    155
Link            155
dtype: int64

In [18]:
data['Link'][154]

'https://uel.unipd.it/en/masters/mgr-mediation-and-restorative-justice/'

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the page
url = 'https://uel.unipd.it/en/masters/psco-school-psychology/'

# Send a request to fetch the page content
response = requests.get(url)

# Parse the page content with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the main content div with class 'wrapper'
main_div = soup.find("body", class_="mat-typography")
if main_div:
    # Remove any header or footer elements inside the main div (if they exist)
    for tag in main_div.find_all(['header', 'footer']):
        tag.decompose()  # Remove these tags and their content

    # Extract the text from the cleaned main div
    text = main_div.get_text(separator='\n', strip=True)
    print(text)

else:
    print("Div with class 'wrapper' not found.")


In [22]:
from bs4 import BeautifulSoup
import requests

# Step 1: Define the URL
url = "https://uel.unipd.it/en/masters/"

# Step 2: Make a request to the URL
response = requests.get(url)
if response.status_code == 200:
    print(f"Successfully fetched the page: {url}")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")
    exit()

# Step 3: Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(response.content, 'html.parser')

# Step 4: Locate the <body> tag with the specific class
body = soup.find('body', class_="master-template-default single single-master postid-4522 fusion-image-hovers fusion-pagination-sizing fusion-button_type-flat fusion-button_span-no fusion-button_gradient-linear avada-image-rollover-circle-yes avada-image-rollover-yes avada-image-rollover-direction-left fusion-body ltr fusion-sticky-header no-mobile-sticky-header no-mobile-slidingbar no-mobile-totop fusion-disable-outline fusion-sub-menu-fade mobile-logo-pos-left layout-wide-mode avada-has-boxed-modal-shadow- layout-scroll-offset-full avada-has-zero-margin-offset-top fusion-top-header menu-text-align-center mobile-menu-design-classic fusion-show-pagination-text fusion-header-layout-v3 avada-responsive avada-footer-fx-none avada-menu-highlight-style-bottombar fusion-search-form-classic fusion-main-menu-search-overlay fusion-avatar-circle avada-sticky-shrinkage avada-dropdown-styles avada-blog-layout-grid avada-blog-archive-layout-grid avada-header-shadow-no avada-menu-icon-position-left avada-has-megamenu-shadow avada-has-mobile-menu-search avada-has-breadcrumb-mobile-hidden avada-has-titlebar-hide avada-header-border-color-full-transparent avada-has-pagination-width_height avada-flyout-menu-direction-fade avada-ec-views-v1 fusion-no-touch do-animate")
print(body)
if body:
    # Extract all text from the body tag
    # Remove any header or footer elements inside the main div (if they exist)
    for tag in main_div.find_all(['header', 'footer']):
        tag.decompose()  # Remove these tags and their content
    
    all_text = body.get_text(separator='\n', strip=True)
    print("Extracted text from the body tag:")
    print(all_text)
   

Successfully fetched the page: https://uel.unipd.it/en/masters/
None


In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()

# Step 2: Open the URL with Selenium
url = "https://uel.unipd.it/en/masters/mgr-mediation-and-restorative-justice/"
driver.get(url)

# Wait for the page to load
time.sleep(5)  # Adjust as necessary for slower connections

# Step 3: Get the page source
page_source = driver.page_source

# Step 4: Parse with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Step 5: Locate the <body> tag
body = soup.find('body')

if body:
    # Exclude header and footer
    header = body.find('header')
    footer = body.find('footer')
    if header:
        header.extract()
    if footer:
        footer.extract()

    # Extract all text from the modified body tag
    all_text = body.get_text(separator='\n', strip=True)
    print("Extracted text (without header and footer):")
    print(all_text)
    
    

Extracted text (without header and footer):
Skip to content
Second level university master
MGR
Mediation and Restorative Justice
Methodology and tools for the application in the familiar, penal, community, organizational and commercial fields
Home
>
Master e corsi
>
Second level university master
>
MGR - Mediation and Restorative Justice
Home
>
Master e corsi
>
Second level university master
>
MGR - Mediation and Restorative Justice
The Master joins the second extraordinary pre-enrolment term from the 4th to the 25th November 2024, 12.30 pm.
The
Master’s in Mediation and Restorative Justice
prepares future mediators – given the growing demand to streamline the civil and criminal process – to refer more to restorative justice and conflict management paradigms in various areas.
The mediator manages the process in which two or more parties define their own, exclusive reality. The aim of the course is to develop
high level of mediation skills
, effective in assisting the parties in reachin

In [29]:

# Search for the phrase "200 and 300-level" in the content
search_phrase = "4.022,50"

# Check if the phrase exists in the file content
if search_phrase in all_text:
    print(f"The phrase '{search_phrase}' was found in the file.")
else:
    print(f"The phrase '{search_phrase}' was NOT found in the file.")

The phrase '4.022,50' was found in the file.


In [30]:
data

,Program Name,Link
0,PSCO\nSchool psychology,https://uel.unipd.it/en/masters/psco-school-ps...
1,ENSAC\nEndocrine surgery,https://uel.unipd.it/en/masters/ensac-endocrin...
2,OMPS\nOral Pathology and Medicine,https://uel.unipd.it/en/masters/omps-oral-path...
3,TUAPP\nLearning Tutor,https://uel.unipd.it/en/masters/tuapp-learning...
4,MHR\nPharmacoepidemiology and Integrated Care ...,https://uel.unipd.it/en/masters/mhr-pharmacoep...
...,...,...
150,CPS\nClinical product specialist and clinical ...,https://uel.unipd.it/en/masters/cps-clinical-p...
151,MFUI\nMyofascial ultrasound imaging,https://uel.unipd.it/en/masters/mfui-myofascia...
152,FACE\nOrthodontics with Straight-Arch Technique,https://uel.unipd.it/en/masters/face-orthodont...
153,FTP\nPharmacoeconomics and personalized treatment,https://uel.unipd.it/en/masters/ftp-pharmacoec...


In [52]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import os

# Create a directory to store the text files
os.makedirs("University of Trento/", exist_ok=True)

# Iterate through each row in the DataFrame
for index, row in data.iterrows():
    degree_name = row['Program Name']
    url = row['Link']

    driver = webdriver.Chrome()
    driver.get(url)
    # Send a GET request to fetch the HTML content of the webpage
    response = requests.get(url)

    if response.status_code == 200:

        # Wait for the page to load
        time.sleep(5)  # Adjust as necessary for slower connections

        # Step 3: Get the page source
        page_source = driver.page_source

        # Step 4: Parse with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        body = soup.find('body')

        if body:
            # Exclude header and footer
            header = body.find('header')
            footer = body.find('footer')
            if header:
                header.extract()
            if footer:
                footer.extract()

            # Extract all text from the modified body tag
            all_text = body.get_text(separator='\n', strip=True)
            print("Extracted text (without header and footer):")
           
            # Create a valid filename by replacing invalid characters
            filename = f"University of Trento/{degree_name.replace('/', '_').replace(' ', '_').replace('|','_')}.txt"

            # Save the link and text content to the file
            with open(filename, "w", encoding="utf-8") as file:
                # Write the link at the top
                file.write(f"Link: {url}\n\n")
                # Write the extracted content below the link
                file.write(all_text)

            print(f"Content successfully saved to '{filename}'.")
        else:
            print(f"Main content not found for {degree_name}.")
    else:
        print(f"Failed to retrieve the webpage for {degree_name}. Status code: {response.status_code}")


Extracted text (without header and footer):
Content successfully saved to 'University of Trento/CAT.txt'.
Extracted text (without header and footer):
Content successfully saved to 'University of Trento/ASSIMedical_Care_for_Ostomated_and_Incontinent_Patients.txt'.
Extracted text (without header and footer):
Content successfully saved to 'University of Trento/LINFOLymphology.txt'.
Extracted text (without header and footer):
Content successfully saved to 'University of Trento/MASAeronautical_and_space_medicine.txt'.
Extracted text (without header and footer):
Content successfully saved to 'University of Trento/BIOETICAPost-Graduate_Course_in_Bioethics.txt'.
Extracted text (without header and footer):
Content successfully saved to 'University of Trento/PMAMedically_assisted_procreation_techniques_and_human_reproductive_sciences.txt'.
Extracted text (without header and footer):
Content successfully saved to 'University of Trento/TAPUAdvanced_ureteral_pathology_treatment_techniques.txt'.
Ext

Extracted text (without header and footer):
Content successfully saved to 'University of Trento/FACEOrthodontics_with_Straight-Arch_Technique.txt'.
Extracted text (without header and footer):
Content successfully saved to 'University of Trento/FTPPharmacoeconomics_and_personalized_treatment.txt'.
Extracted text (without header and footer):
Content successfully saved to 'University of Trento/MGRMediation_and_Restorative_Justice.txt'.


In [39]:
data["Program Name"] = data["Program Name"].str.replace("\n", "", regex=False)

In [48]:
data.iloc[97,0]='CAT'

In [51]:
data=data.iloc[97:]

In [54]:
data

,Program Name,Link
97,CAT,https://uel.unipd.it/en/masters/cat-creative-a...
98,ASSIMedical Care for Ostomated and Incontinent...,https://uel.unipd.it/en/masters/assi-medical-c...
99,LINFOLymphology,https://uel.unipd.it/en/masters/linfo-lymphology/
100,MASAeronautical and space medicine,https://uel.unipd.it/en/masters/mas-aeronautic...
101,BIOETICAPost-Graduate Course in Bioethics,https://uel.unipd.it/en/masters/bioetica-post-...
102,PMAMedically assisted procreation techniques a...,https://uel.unipd.it/en/masters/pma-medically-...
103,TAPUAdvanced ureteral pathology treatment tech...,https://uel.unipd.it/en/masters/tapu-advanced-...
104,OSRICHospital dentistry and clinical research,https://uel.unipd.it/en/masters/osric-hospital...
105,"MIEEuropean policies, projects and funding",https://uel.unipd.it/en/masters/mie-european-p...
106,PPEPaediatric psychology (0-6),https://uel.unipd.it/en/masters/ppe-paediatric...
